In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [2]:
# import env file for hostname, username, password, and db_name
from env import host, user, password, db_name

In [3]:
# Pass env file authentication to container 'url'
url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [4]:
# define sql search for all records from all tables in Zillow database
sql ='''
SELECT *
FROM properties_2017 AS prop17
JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN predictions_2017 as pred_17 USING (parcelid)
LEFT JOIN airconditioningtype USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype USING (buildingclasstypeid)
LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
LEFT JOIN storytype USING (storytypeid)
LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
LEFT JOIN unique_properties USING (parcelid)
WHERE transactiondate < '2018-01-01' 
AND propertylandusetypeid='261'
    '''

In [32]:
# load zillow data from saved csv or pull from sql server and save to csv
import os
file = 'zillow_data.csv'
if os.path.isfile(file):
    df = pd.read_csv(file, index_col=0)
else:
    df = pd.read_sql(sql,url)
    df.to_csv(file)

In [34]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,propertylandusetypeid,id,basementsqft,...,censustractandblock,propertylandusedesc,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,NaN,NaN,NaN,NaN,261.0,1727539,NaN,...,6.059063e+13,Single Family Residential,0.025595,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,17052889,NaN,NaN,NaN,NaN,NaN,NaN,261.0,1387261,NaN,...,6.111001e+13,Single Family Residential,0.055619,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN
2,14186244,NaN,NaN,NaN,NaN,NaN,NaN,261.0,11677,NaN,...,6.059022e+13,Single Family Residential,0.005383,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN
3,12177905,NaN,NaN,2.0,NaN,NaN,NaN,261.0,2288172,NaN,...,6.037300e+13,Single Family Residential,-0.103410,2017-01-01,NaN,NaN,NaN,Central,NaN,NaN
4,12095076,NaN,NaN,2.0,NaN,NaN,1.0,261.0,781532,NaN,...,6.037461e+13,Single Family Residential,-0.001011,2017-01-01,Central,NaN,NaN,Central,NaN,NaN


In [33]:
# df_3 shape
df.shape

(52438, 68)

In [7]:
df.parcelid.duplicated().sum()

118

In [8]:
# Sort df by parcelid and then by transaction date to group by parcelid and to make sure transaction date is last
df = df.sort_values(by=['parcelid','transactiondate'])
df.head()

,parcelid,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,propertylandusetypeid,id,basementsqft,...,censustractandblock,propertylandusedesc,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
37223,10711855,NaN,NaN,2.0,NaN,NaN,NaN,261.0,1087254,NaN,...,6.037113e+13,Single Family Residential,-0.007357,2017-07-07,NaN,NaN,NaN,Central,NaN,NaN
48246,10711877,NaN,NaN,2.0,NaN,NaN,1.0,261.0,1072280,NaN,...,6.037113e+13,Single Family Residential,0.021066,2017-08-29,Central,NaN,NaN,Central,NaN,NaN
15536,10711888,NaN,NaN,2.0,NaN,NaN,1.0,261.0,1340933,NaN,...,6.037113e+13,Single Family Residential,0.077174,2017-04-04,Central,NaN,NaN,Central,NaN,NaN
12106,10711910,NaN,NaN,2.0,NaN,NaN,NaN,261.0,1878109,NaN,...,6.037113e+13,Single Family Residential,-0.041238,2017-03-17,NaN,NaN,NaN,Central,NaN,NaN
13666,10711923,NaN,NaN,2.0,NaN,NaN,NaN,261.0,2190858,NaN,...,6.037113e+13,Single Family Residential,-0.009496,2017-03-24,NaN,NaN,NaN,Central,NaN,NaN


In [9]:
df = df.drop_duplicates(subset='parcelid', keep='last')

In [10]:
df.parcelid.duplicated().sum()

0

In [11]:
df.shape

(52320, 68)

In [12]:
# Replace blank values with NaN
df = df.replace('',np.nan)

In [13]:
# check if long/lat columns have nulls
df.longitude.isnull().sum(), df.latitude.isnull().sum()

(0, 0)

***

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [15]:
# Describe without scientific notation
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,parcelid,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,propertylandusetypeid,id,basementsqft,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror,buildingclassdesc
count,52320,76,47,33850,0,70,13615,52320,52320,47,...,81,52238,52319,52320,52319,52316,2068,52199,52320,0
mean,1.29968e+07,5.97368,7,3.96561,nan,7.1,2.43959,261,1.49691e+06,678.979,...,1,196636,529824,2016,333492,6454.74,14.1011,6.05024e+13,0.0181378,nan
std,3.35092e+06,0.229416,0,2.56266,nan,2.66567,3.84793,0,859433,711.825,...,0,254286,751830,0,570511,8752.48,2.4004,1.86113e+12,0.176903,nan
min,1.07119e+07,4,7,1,nan,2,1,261,349,38,...,1,129,1000,2016,161,49.18,4,6.0371e+13,-4.65542,nan
25%,1.15102e+07,6,7,2,nan,7,1,261,757614,263.5,...,1,77159,194033,2016,76194,2660.98,14,6.0374e+13,-0.0247003,nan
50%,1.25783e+07,6,7,2,nan,7,1,261,1.50013e+06,512,...,1,131905,374006,2016,218079,4650.57,15,6.03762e+13,0.00694008,nan
75%,1.41304e+07,6,7,7,nan,7,1,261,2.24133e+06,809.5,...,1,226453,619354,2016,408777,7379.27,15,6.05904e+13,0.0406021,nan
max,1.67688e+08,6,7,24,nan,21,13,261,2.98227e+06,3560,...,1,9.1649e+06,4.90612e+07,2016,4.89522e+07,586639,99,4.8303e+14,5.263,nan


In [16]:
df.shape

(52320, 68)

In [17]:
# df.hist()
# plt.tight_layout();

In [18]:
# for col in df.columns:
#     df[col].hist()

In [19]:
# for col in df.columns:
#     print(f'_-------------- {col} ---------------')
#     print(df[col].value_counts())

In [20]:
#df.info(verbose=True)

***

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [21]:
# show nulls per column
df.isna().sum()

parcelid                      0
typeconstructiontypeid    52244
storytypeid               52273
heatingorsystemtypeid     18470
buildingclasstypeid       52320
                          ...  
architecturalstyledesc    52250
buildingclassdesc         52320
heatingorsystemdesc       18470
storydesc                 52273
typeconstructiondesc      52244
Length: 68, dtype: int64

In [40]:
# show nulls per column
df.isna().mean()

parcelid                  0.000000
typeconstructiontypeid    0.998551
storytypeid               0.999104
heatingorsystemtypeid     0.352874
buildingclasstypeid       1.000000
                            ...   
architecturalstyledesc    0.998665
buildingclassdesc         1.000000
heatingorsystemdesc       0.352874
storydesc                 0.999104
typeconstructiondesc      0.998551
Length: 68, dtype: float64

In [43]:
col_null = pd.DataFrame()
col_null['columns_name'] = df.isna().sum().index
col_null

,columns_name
0,parcelid
1,typeconstructiontypeid
2,storytypeid
3,heatingorsystemtypeid
4,buildingclasstypeid
...,...
64,buildingclassdesc
65,heatingorsystemdesc
66,storydesc
67,typeconstructiondesc


In [44]:
col_null['null_count'] = df.isna().sum().values
col_null

In [45]:
col_null['null_percent'] = df.isna().mean().values
col_null

,columns_name,null_count,null_percent
0,parcelid,0,0.000000
1,typeconstructiontypeid,52362,0.998551
2,storytypeid,52391,0.999104
3,heatingorsystemtypeid,18504,0.352874
4,buildingclasstypeid,52438,1.000000
...,...,...,...
64,buildingclassdesc,52438,1.000000
65,heatingorsystemdesc,18504,0.352874
66,storydesc,52391,0.999104
67,typeconstructiondesc,52362,0.998551


1. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [48]:
# show the number of columns missing from each row
df.isna().sum(axis=1)

0        36
1        33
2        34
3        32
4        29
         ..
52433    34
52434    33
52435    32
52436    32
52437    34
Length: 52438, dtype: int64

In [49]:
# show the percent of columns missing from each row
df.isna().mean(axis=1)

0        0.521739
1        0.478261
2        0.492754
3        0.463768
4        0.420290
           ...   
52433    0.492754
52434    0.478261
52435    0.463768
52436    0.463768
52437    0.492754
Length: 52438, dtype: float64

In [52]:
# show the amount of rows missing specific number of column
df.isna().sum(axis=1).value_counts()

33    11995
34    11180
32     8901
31     6004
36     4147
35     3478
29     2531
30     2204
37     1023
28      392
38      227
27      177
44       47
26       30
39       29
40       15
24       12
25       11
43       10
42        7
45        6
41        6
23        2
48        2
46        1
47        1
dtype: int64

In [54]:
row_nulls = pd.DataFrame()
row_nulls['num_cols_missing'] = df.isna().sum().values
row_nulls

,num_cols_missing
0,0
1,52362
2,52391
3,18504
4,52438
...,...
64,52438
65,18504
66,52391
67,52362


In [55]:
row_nulls['percent_cols_missing'] = df.isna().mean().values
row_nulls

,num_cols_missing,percent_cols_missing
0,0,0.000000
1,52362,0.998551
2,52391,0.999104
3,18504,0.352874
4,52438,1.000000
...,...,...
64,52438,1.000000
65,18504,0.352874
66,52391,0.999104
67,52362,0.998551


In [60]:
df.isna().sum(axis=1).value_counts().values

array([11995, 11180,  8901,  6004,  4147,  3478,  2531,  2204,  1023,
         392,   227,   177,    47,    30,    29,    15,    12,    11,
          10,     7,     6,     6,     2,     2,     1,     1])

In [57]:
df.isna().sum(axis=1).value_counts()

33    11995
34    11180
32     8901
31     6004
36     4147
35     3478
29     2531
30     2204
37     1023
28      392
38      227
27      177
44       47
26       30
39       29
40       15
24       12
25       11
43       10
42        7
45        6
41        6
23        2
48        2
46        1
47        1
dtype: int64

In [56]:
row_nulls['num_row'] = df.isna().sum(axis=1).value_counts().values
row_nulls

ValueError: Length of values (26) does not match length of index (69)

In [53]:
row_nulls = pd.DataFrame()
row_nulls['num_cols_missing'] = df.isna().sum().values
row_nulls['percent_cols_missing'] = df.isna().mean().values
row_nulls['num_row'] = df.isna().sum(axis=1).value_counts().values
row_nulls

ValueError: Length of values (26) does not match length of index (69)

In [23]:
# transpose to make each row an attribute name
df = df.T
df.head()

,37223,48246,15536,12106,13666,4381,36294,10462,42662,7623,...,8832,6809,49219,37075,2794,42306,31148,23911,9001,28113
parcelid,10711855,10711877,10711888,10711910,10711923,10711945,10711956,10711995,10712005,10712007,...,167636496,167636512,167636726,167636727,167637054,167637371,167638003,167639152,167686999,167687839
typeconstructiontypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storytypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
heatingorsystemtypeid,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,7.0,2.0,2.0,2.0,NaN,NaN
buildingclasstypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#df.info(verbose=True)

In [25]:
# Add new column to of NaN sums to column position 0
df.insert(0, 'null_count', df.isnull().sum(axis=1))

In [26]:
df.head()

,null_count,37223,48246,15536,12106,13666,4381,36294,10462,42662,...,8832,6809,49219,37075,2794,42306,31148,23911,9001,28113
parcelid,0,10711855,10711877,10711888,10711910,10711923,10711945,10711956,10711995,10712005,...,167636496,167636512,167636726,167636727,167637054,167637371,167638003,167639152,167686999,167687839
typeconstructiontypeid,52244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storytypeid,52273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
heatingorsystemtypeid,18470,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,7.0,2.0,2.0,2.0,NaN,NaN
buildingclasstypeid,52320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Make second column as percent of total rows that have missing values for that attribute.
df.insert(1, 'null_percent', round((df.isnull().sum(axis=1) / df.shape[1])*100))
df.head()

,null_count,null_percent,37223,48246,15536,12106,13666,4381,36294,10462,...,8832,6809,49219,37075,2794,42306,31148,23911,9001,28113
parcelid,0,0.0,10711855,10711877,10711888,10711910,10711923,10711945,10711956,10711995,...,167636496,167636512,167636726,167636727,167637054,167637371,167638003,167639152,167686999,167687839
typeconstructiontypeid,52244,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storytypeid,52273,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
heatingorsystemtypeid,18470,35.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,7.0,2.0,2.0,2.0,NaN,NaN
buildingclasstypeid,52320,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# show number of columsn with percent of nulls
df.null_percent.value_counts()

0.0      27
100.0    14
35.0      4
96.0      3
92.0      2
74.0      2
99.0      2
66.0      2
98.0      2
64.0      1
79.0      1
72.0      1
97.0      1
81.0      1
36.0      1
86.0      1
2.0       1
87.0      1
1.0       1
Name: null_percent, dtype: int64

### Notes:
- 14 columsn have all nulls

***

### 1. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

***